# Maximize Profit with K transaction -- Solved by Dynamic Programming

<img src="img/Maximize Profit.png" height=50>

## General Idea:
Set max profit in hash table to 0 when k = 0<br>
When k > 0 <br>
<br>max_profit_today = |--the max profit in this period when k = k -1-- | --the max profit from (last sell date+1) to today|



In [3]:
prices = [5, 11, 3, 50, 60, 90]
k = 2
def maxProfitWithKTransactions(prices, k):
    hash_table = {0: [0]*len(prices)}
    for i in range(1, k+1):
        hash_table[i] = [0]
        for date in range(1, len(prices)):
            last_day_max_profit = hash_table[i][date-1]
            max_profit = last_day_max_profit
            for last_sell_date in range(0, date):
                last_sell_date_max_profit = hash_table[i-1][last_sell_date] + prices[date] - prices[last_sell_date]
                max_profit = max(max_profit, last_sell_date_max_profit)
            hash_table[i].append(max_profit)
    return max_profit
        
maxProfitWithKTransactions(prices, k)

93

# Find All Substring


In [16]:
import re
s: str = "ABCABCABCABCABC"
pat: str = "(?=ABCA)"
    
[x.start() for x in re.finditer(pat, s)]

[0, 3, 6, 9]

# Palindrome Pairs
https://leetcode.com/articles/palindrome-pairs/ <br>
Given a list of unique words, find all pairs of distinct indices (i, j) in the given list, so that the concatenation of the two words, i.e. words[i] + words[j] is a palindrome.

Example:

**Input**: ["abcd","dcba","lls","s","sssll"] <br>
**Output**: [[0,1],[1,0],[3,2],[2,4]] <br>
**Explanation**: The palindromes are ["dcbaabcd","abcddcba","slls","llssssll"]

## - Solution
<img src="img/Palindrom pair 3 cases.png" height=50>

Key points:
- We save words reversely so that we could find palindrome easily
- The node is marked if this is the end of a word
- Trie detects suffixes which are palindrome and stored the word index to the level where the subffix start. This helps if we could find a reverse word end at the start of the subffix, then the two words is a palindrome pair

In [5]:
class TrieNode:
    def __init__(self):
        self.next = collections.defaultdict(TrieNode)
        self.ending_word = -1
        self.palindrome_suffixes = []

class Solution:
    def palindromePairs(self, words):

        # Create the Trie and add the reverses of all the words.
        trie = TrieNode()
        for i, word in enumerate(words):
            word = word[::-1] # We want to insert the reverse.
            current_level = trie
            for j, c in enumerate(word):
                # Check if remainder of word is a palindrome.
                if word[j:] == word[j:][::-1]:# Is the word the same as its reverse?
                    current_level.palindrome_suffixes.append(i)
                # Move down the trie.
                current_level = current_level.next[c]
            current_level.ending_word = i

        # Look up each word in the Trie and find palindrome pairs.
        solutions = []
        for i, word in enumerate(words):
            current_level = trie
            for j, c in enumerate(word):
                # Check for case 3.
                if current_level.ending_word != -1:
                    if word[j:] == word[j:][::-1]: # Is the word the same as its reverse?
                        solutions.append([i, current_level.ending_word])
                if c not in current_level.next:
                    break
                current_level = current_level.next[c]
            else: # Case 1 and 2 only come up if whole word was iterated.
                # Check for case 1.
                if current_level.ending_word != -1 and current_level.ending_word != i:
                    solutions.append([i, current_level.ending_word])
                # Check for case 2.
                for j in current_level.palindrome_suffixes:
                    solutions.append([i, j])
        return solutions
    
    
import collections
s = Solution()
s.palindromePairs(["cat", "solostac"])

[[0, 1]]

# Meeting Room Schedule (Heap)

Given an array of meeting time intervals consisting of start and end times [[s1,e1],[s2,e2],...] (si < ei), find the minimum number of conference rooms required. <br>

Example 1: <br>
Input: [[0, 30],[5, 10],[15, 20]] <br>
Output: 2 <br><br>

Example 2:<br>
Input: [[7,10],[2,4]]<br>
Output: 1<br>

Program workflow<br>
1. sort intervals by starting time <br>
2. add the initial meeting on the heap, use the end time as key <br>
3. before trying to add new meeting, pop the meeting top if the meeting has ended <br>
4. push the new meeting

In [28]:
import heapq

intervals = [[13,15],[1,13]]
start = 0
end = 1
class Solution:
    def minMeetingRooms(self, intervals: list) -> int:
        if len(intervals) == 0:
            return 0
        intervals = sorted(intervals)
        heap = []
        heapq.heappush(heap, intervals[0][end])
        max_len = 1
        # the key of heap is the end time of meetings
        for m in intervals[1:]:
            print(m)
            while(len(heap) > 0 and m[start] >= heap[0]):
                heapq.heappop(heap)
            heapq.heappush(heap, m[end])
            max_len = len(heap) if max_len < len(heap) else max_len
        return max_len
    
s = Solution()
s.minMeetingRooms(intervals)

[13, 15]


1

# Ugly Value
Write a program to find the n-th ugly number. <br>
Ugly numbers are positive numbers whose prime factors only include 2, 3, 5.  <br><br>

Example:<br>
Input: n = 10 <br>
Output: 12 <br>
Explanation: 1, 2, 3, 4, 5, 6, 8, 9, 10, 12 is the sequence of the first 10 ugly numbers. <br>

Video Solution: https://leetcode.com/problems/ugly-number-ii/solution/

## Heap Solution

In [32]:
import copy
class Solution:
    def nthUglyNumber(self, n: int) -> int:
        # for n in range(6, 100):
        heap = [1]
        seen = {1, }
        pv = 1
        while(n>0):
            pv = heapq.heappop(heap)
            n -= 1
            for f in [2, 3, 5]:
                if pv*f not in seen:
                    heapq.heappush(heap, pv*f)
                    seen.add(pv*f)
        return pv

s = Solution()
s.nthUglyNumber(10)

12

## Dynamic Programming

In [33]:
import numpy as np
class Solution:
    def nthUglyNumber(self, n: int) -> int:
        # for n in range(6, 100):
        u_nums = [1]
        n -= 1
        i2 = i3 = i5 = 0
        while n > 0:
            tmp = [u_nums[i2]*2, u_nums[i3]*3, u_nums[i5]*5]
            u_nums.append(min(tmp))
            n -= 1
            if tmp[0] == min(tmp): i2 += 1
            if tmp[1] == min(tmp): i3 += 1
            if tmp[2] == min(tmp): i5 += 1
        return u_nums[-1]

s = Solution()
s.nthUglyNumber(10)

12

# Two Pointer + Hash map
Longest Substring with At Most K Distinct Characters<br>
https://leetcode.com/problems/longest-substring-with-at-most-k-distinct-characters/<br>
This was a interview question from ultimate, I failed to answer it<br>
Given a string, find the length of the longest substring T that contains at most k distinct characters.<br>

Example 1:<br><br>

Input: s = "eceba", k = 2<br>
Output: 3<br>
Explanation: T is "ece" which its length is 3.<br>
Example 2:<br>
<br>
Input: s = "aa", k = 1<br>
Output: 2<br>
Explanation: T is "aa" which its length is 2.<br>

In [3]:
class Solution:  
    def lengthOfLongestSubstringKDistinct(self, s: str, k: int) -> int:
        if s == "" or k == 0: return 0
        if len(s) == 1: return 1
        l = 0
        r = 0
        max_len = 0
        hm = {}
        while(r < len(s)):
            hm[s[r]] = r
            r += 1
            if len(hm.keys()) > k:
                min_index = min(hm.values())
                del hm[s[min_index]]
                l = min_index + 1
            max_len = max(r-l, max_len)
            print(hm, max_len)
            
        return max_len
    
s = Solution()
s.lengthOfLongestSubstringKDistinct("eceab", 2)

{'e': 0} 1
{'e': 0, 'c': 1} 2
{'e': 2, 'c': 1} 3
{'e': 2, 'a': 3} 3
{'a': 3, 'b': 4} 3


3